In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Race Results

In [4]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [69]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP=GP[GP["Position"]!=0]
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
1,2,,,,Brabham,,0,Teo,Fabi,Teo Fabi,0,0,4,0,7,NaT,NaT,NaT,0 days 01:22:09.163000,Finished,3,1984,Austrian Grand Prix
2,2,,,,Brabham,,0,Teo,Fabi,Teo Fabi,0,0,3,0,23,NaT,NaT,NaT,0 days 01:57:08.370000,Finished,4,1984,Detroit Grand Prix
3,71,,,,Kurtis Kraft,,0,Al,Herman,Al Herman,0,0,7,0,16,NaT,NaT,NaT,0 days 04:00:23.770000,Finished,0,1955,Indianapolis 500
4,89,,,,Dunn,,0,Al,Herman,Al Herman,0,0,13,0,23,NaT,NaT,NaT,0 days 03:47:29.600000,Finished,0,1959,Indianapolis 500
5,52,,,,Kurtis Kraft,,0,Al,Keller,Al Keller,0,0,11,0,21,NaT,NaT,NaT,0 days 03:53:28,Finished,0,1958,Indianapolis 500


In [70]:
year=2024

In [71]:
grouped = GP.groupby(['Year','Abbreviation','Position'])["Points"].agg('count').reset_index()
grouped.columns=['Year','Abbreviation','Position','Count']
grouped=grouped[grouped['Year']==year]
grouped=grouped.sort_values(by=['Position','Count'], ascending=[True,False])
grouped.head()

,Year,Abbreviation,Position,Count
6316,2020,HAM,1,11
6276,2020,BOT,1,2
6443,2020,VER,1,2
6287,2020,GAS,1,1
6386,2020,PER,1,1


In [72]:
fig=px.bar(grouped, 
           x="Position", 
           y='Count', 
           color='Abbreviation',
        #    title="Race Position Counts By Driver for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5)
)
# fig.update_traces(marker={'size': 12})

In [73]:
fig.write_html("Race Final Position Counts By Driver.html".format(year))

In [74]:
#By Team

In [75]:
grouped_team = GP.groupby(['Year','TeamName','Position'])["Points"].agg('count').reset_index()
grouped_team.columns=['Year','Constructor','Position','Count']
grouped_team=grouped_team[grouped_team['Year']==year]
grouped_team=grouped_team.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_team.head()

,Year,Constructor,Position,Count
12221,2020,Mercedes,1,13
12246,2020,Red Bull Racing,1,2
12165,2020,AlphaTauri,1,1
12232,2020,Racing Point,1,1
12222,2020,Mercedes,2,7


In [76]:
fig_team=px.bar(grouped_team, 
           x="Position", 
           y='Count', 
           color='Constructor',
        #    title="Race Position Counts By Team for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            },
            )

fig_team.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)

# fig.update_traces(marker={'size': 12})

In [77]:
fig_team.write_html("Race Final Position Counts By Team.html".format(year))

In [78]:
#Qualy

In [79]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_qualifying`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2_qualy = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [80]:
GP_qualy=df_bq2_qualy
GP_qualy=GP_qualy.fillna(0)
GP_qualy["Position"]=GP_qualy["Position"].astype(int)
GP_qualy["GridPosition"]=GP_qualy["GridPosition"].astype(int)
GP_qualy["Points"]=GP_qualy["Points"].astype(int)
GP_qualy=GP_qualy[GP_qualy["Position"]!=0]
GP_qualy.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,21,0,ALO,0,Minardi,0,0,Fernando,Alonso,Fernando Alonso,0,0,17,17,17,0 days 00:01:13.991000,0,0,0,0,0,2001,United States Grand Prix
1,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,12,12,12,0 days 00:01:20.543000,0,0,0,0,0,1995,Hungarian Grand Prix
2,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,13,13,13,0 days 00:01:57.167000,0,0,0,0,0,1995,Argentine Grand Prix
3,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,15,15,15,0,0,0,0,0,0,1995,Australian Grand Prix
4,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,16,16,16,0 days 00:01:49.302000,0,0,0,0,0,1995,German Grand Prix


In [81]:
grouped_qualy = GP_qualy.groupby(['Year','Abbreviation','Position'])["Points"].agg('count').reset_index()
grouped_qualy.columns=['Year','Abbreviation','Position','Count']
grouped_qualy=grouped_qualy[grouped_qualy['Year']==year]
grouped_qualy=grouped_qualy.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_qualy.head()

,Year,Abbreviation,Position,Count
4163,2020,HAM,1,10
4133,2020,BOT,1,5
4261,2020,STR,1,1
4272,2020,VER,1,1
4134,2020,BOT,2,9


In [82]:
fig_qualy=px.bar(grouped_qualy, 
           x="Position", 
           y='Count', 
           color='Abbreviation',
        #    title="Race Position Counts By Driver for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )

fig_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5)
)
# fig.update_traces(marker={'size': 12})

In [83]:
fig_qualy.write_html("Qualifying Final Position Counts By Driver.html".format(year))

In [84]:
#By Team

In [85]:
grouped_team_qualy = GP_qualy.groupby(['Year','TeamName','Position'])["Points"].agg('count').reset_index()
grouped_team_qualy.columns=['Year','Constructor','Position','Count']
grouped_team_qualy=grouped_team_qualy[grouped_team_qualy['Year']==year]
grouped_team_qualy=grouped_team_qualy.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_team_qualy.head()

,Year,Constructor,Position,Count
2875,2020,Mercedes,1,15
2881,2020,Racing Point,1,1
2896,2020,Red Bull Racing,1,1
2876,2020,Mercedes,2,14
2897,2020,Red Bull Racing,2,3


In [86]:
fig_team_qualy=px.bar(grouped_team_qualy, 
           x="Position", 
           y='Count', 
           color='Constructor',
        #    title="Race Position Counts By Team for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            },
            )

fig_team_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)

# fig.update_traces(marker={'size': 12})

In [87]:
fig_team_qualy.write_html("Qualifying Final Position Counts By Team.html".format(year))